# Projeto Final - Olist

## Descrição

"Conjunto de dados públicos de comércio eletrônico brasileiro por Olist
O conjunto de dados tem informações de 100 mil pedidos de 2016 a 2018 feitos em vários marketplaces no Brasil.

A Olist conecta pequenas empresas de todo o Brasil a canais sem complicações e com um único contrato. Esses comerciantes podem vender seus produtos através da Olist Store e enviá-los diretamente aos clientes usando os parceiros de logística da Olist.

Depois que um cliente compra o produto da Olist Store, um vendedor é notificado para atender esse pedido. Assim que o cliente recebe o produto, ou vence a data prevista de entrega, o cliente recebe uma pesquisa de satisfação por e-mail onde pode dar uma nota da experiência de compra e anotar alguns comentários."


## Importando Bibliotecas

In [3]:
import pandas as pd
import numpy as np
import os
import re
from sklearn.linear_model import LinearRegression
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from pycaret.classification import *

# Leitura arquivos

In [1]:
itens = pd.read_csv("data\itens.csv") #items
ordens = pd.read_csv("data\ordens.csv") #order
avaliacoes = pd.read_csv("data\olist_order_reviews_dataset.csv") #reviews
geolocal = pd.read_csv("data\olist_geolocation_dataset.csv") #geolocation
vendedores = pd.read_csv("data\olist_sellers_dataset.csv") #seller
pagamentos = pd.read_csv("data\olist_order_payments_dataset.csv") #payments
clientes = pd.read_csv("data\clientes.csv") #customers
produtos = pd.read_csv("data\olist_products_dataset.csv") #products

NameError: name 'pd' is not defined

## Leitura dos arquivos na pasta

In [4]:
caminho = r'C:\Users\USER\Desktop\Ironhack\Github\Projeto Final - Olist\Data'
lista = []
df = pd.DataFrame()

for f in os.listdir (caminho + "/"):
    if f != 'product_category_name_translation.csv':
        frames = pd.read_csv(caminho + "/" + f)
    df = df.append(frames, ignore_index= True)
df.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,payment_value,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409.0,franca,SP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790.0,sao bernardo do campo,SP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151.0,sao paulo,SP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775.0,mogi das cruzes,SP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056.0,campinas,SP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Verificando dados

In [48]:
df.isna().sum()

customer_id                      1355064
customer_unique_id               1454505
customer_zip_code_prefix         1454505
customer_city                    1454505
customer_state                   1454505
geolocation_zip_code_prefix       553783
geolocation_lat                   553783
geolocation_lng                   553783
geolocation_city                  553783
geolocation_state                 553783
order_id                         1138745
order_status                     1454505
order_purchase_timestamp         1454505
order_approved_at                1454665
order_delivered_carrier_date     1456288
order_delivered_customer_date    1457470
order_estimated_delivery_date    1454505
order_item_id                    1441296
product_id                       1408345
seller_id                        1435106
shipping_limit_date              1441296
price                            1441296
freight_value                    1441296
payment_sequential               1450060
payment_type    

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1553946 entries, 0 to 1553945
Data columns (total 44 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   customer_id                    198882 non-null   object 
 1   customer_unique_id             99441 non-null    object 
 2   customer_zip_code_prefix       99441 non-null    float64
 3   customer_city                  99441 non-null    object 
 4   customer_state                 99441 non-null    object 
 5   geolocation_zip_code_prefix    1000163 non-null  float64
 6   geolocation_lat                1000163 non-null  float64
 7   geolocation_lng                1000163 non-null  float64
 8   geolocation_city               1000163 non-null  object 
 9   geolocation_state              1000163 non-null  object 
 10  order_id                       415201 non-null   object 
 11  order_status                   99441 non-null    object 
 12  order_purchase

In [1]:
df.describe().transpose()

NameError: name 'df' is not defined

## Analisando dados ausentes

In [60]:
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing.head()

,Total,Percent
seller_state,1547756,0.996017
seller_city,1547756,0.996017
seller_zip_code_prefix,1547756,0.996017
review_comment_title,1542378,0.992556
product_photos_qty,1521605,0.979188


## Fazendo Drop na coluna

In [61]:
df.drop(['seller_state', 'seller_city', 'seller_zip_code_prefix', 'review_comment_title', 'product_photos_qty'],axis=1, inplace=True)

Opção para drop em colunas com percentual de numeros ausentes superior a 97% dos dados

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1553946 entries, 0 to 1553945
Data columns (total 39 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   customer_id                    198882 non-null   object 
 1   customer_unique_id             99441 non-null    object 
 2   customer_zip_code_prefix       99441 non-null    float64
 3   customer_city                  99441 non-null    object 
 4   customer_state                 99441 non-null    object 
 5   geolocation_zip_code_prefix    1000163 non-null  float64
 6   geolocation_lat                1000163 non-null  float64
 7   geolocation_lng                1000163 non-null  float64
 8   geolocation_city               1000163 non-null  object 
 9   geolocation_state              1000163 non-null  object 
 10  order_id                       415201 non-null   object 
 11  order_status                   99441 non-null    object 
 12  order_purchase

## Pipeline

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
import sklearn
from sklearn.preprocessing import StandardScaler

In [17]:
scaler = StandardScaler()

## Localização

In [ ]:
from opencage.geocoder import OpenCageGeocode
